<a href="https://colab.research.google.com/github/chetools/CHE4061_Fall2025/blob/main/PonchonSavarit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.5 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import scipy as sp
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_dark"
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)
eps =  np.finfo(np.float64).eps

#Bubble/Dew Point Calculations

In [3]:
p=Props(['Isopropanol','Water'])
R=8.314 #J/(mol K)

In [4]:
z=np.array([0.4,0.6])

In [5]:
def bubbleP(x, T):
    Pi = x*p.Pvap(T)
    P=np.sum(Pi)
    return P, Pi/P

In [6]:
def dewP(y, T):
    Pisat = p.Pvap(T)
    P=1/np.sum(y/Pisat)
    return P, y*P/Pisat

In [7]:
xs= np.linspace(0,1,21)
T=350.
Ps = []
y1s = []
for x in np.c_[xs , 1-xs ]:
    P, y = bubbleP(x, T)
    Ps.append(P)
    y1s.append(y[0])
Ps = np.r_[Ps]
y1s=np.r_[y1s]

In [8]:
fig = make_subplots()
fig.add_scatter(x=xs,y=Ps, name='bubble', mode='lines')
fig.add_scatter(x=y1s,y=Ps, name='dew', mode='lines')
fig.update_layout(width=800,height=600, title=f'Benzene-Tolune VLE at T={T} K', xaxis_title='x, y', yaxis_title='Pressure (Pa)')

In [9]:
def T_estimate(P, x):
    return np.sum(x*1/(1/p.Tbn-R*np.log(P/101325)/p.HvapNB))

In [10]:
def bubbleT(x, P):
    T = sp.optimize.root_scalar(lambda T: bubbleP(x, T)[0] - P, x0=T_estimate(P,x), method='Newton').root

    return T, x*p.Pvap(T)/P

In [11]:
def dewT(y, P):
    T = sp.optimize.root_scalar(lambda T: dewP(y, T)[0] - P, x0=T_estimate(P,y), method='Newton').root

    return T, y*P/p.Pvap(T)


In [12]:
xs= np.linspace(0,1,21)
Ts=[]
P=101325
y1s = []
for x in np.c_[xs , 1-xs ]:
    T, y = bubbleT(x, P)
    Ts.append(T)
    y1s.append(y[0])
Ts = np.r_[Ts]
y1s=np.r_[y1s]
fig = make_subplots(rows=1, cols=2)
fig.add_scatter(x=xs,y=Ts, name='bubble', mode='lines', row=1,col=1)
fig.add_scatter(x=y1s,y=Ts, name='dew', mode='lines', row=1,col=1)
fig.add_scatter(x=xs,y=y1s, mode='lines', row=1,col=2, name='y1')
fig.add_scatter(x=[0,1],y=[0,1], mode='lines', row=1,col=2, name='', line_color='grey')
fig.update_layout(width=800,height=400, title=f'Isopropanol-Water "Ideal" VLE at P={P} Pa', xaxis_title='x, y', yaxis_title='Temperature (K)')

In [13]:
P = 101325. #Pa
T = (dewT(z, P)[0] + bubbleT(z, P)[0])/2

def rachford_rice(z, P, T):
    K = p.Pvap(T)/P
    def rr0(phi):
        return np.sum(z/(1+phi*(K-1))) - 1
    phi = sp.optimize.root_scalar(rr0, x0 = 0.5, method = 'newton').root
    x = z/(1+phi*(K-1))
    y = K*x
    return (phi, x, y)

In [14]:
def bubbleP_NRTL(x, T):
    g = p.NRTL_gamma(x, T)
    pi = x*g*p.Pvap(T)
    P = jnp.sum(pi)
    return P, pi/P

In [15]:
def bubbleT_NRTL(x, P):
    T = sp.optimize.root_scalar(lambda T: bubbleP_NRTL(x, T)[0] - P, x0=bubbleT(x,P)[0], method='Newton').root
    g = p.NRTL_gamma(x, T)
    return T, x*g*p.Pvap(T)/P

In [16]:
def dewP_NRTL(y, T):

    def eq(vec):
        x = vec[:-1]
        P = vec[-1]
        return np.r_[x*p.NRTL_gamma(x, T)*p.Pvap(T)/P - y , np.sum(x)-1.]

    P, x = dewP(y, T)  #assume ideal system for initial guess
    res=sp.optimize.root(eq, np.r_[x, P])
    x = res.x[:-1]
    P = res.x[-1]
    return P, x


In [17]:
def dewT_NRTL(y, P):

    def eq(vec):
        x = vec[:-1]
        T = vec[-1]
        return np.r_[x*p.NRTL_gamma(x, T)*p.Pvap(T)/P - y , np.sum(x)-1.]

    T, x = dewT(y, P)  #assume ideal system for initial guess
    res=sp.optimize.root(eq, np.r_[x, T])
    x = res.x[:-1]
    T = res.x[-1]
    return T, x

In [18]:
xs= np.linspace(0,1,101)
Ts=[]
P=101325.
y1s = []
for x in np.c_[xs , 1-xs ]:
    T, y = bubbleT_NRTL(x, P)
    Ts.append(T)
    y1s.append(y[0])
Ts = np.r_[Ts]
y1s=np.r_[y1s]
fig = make_subplots(rows=1, cols=2)
fig.add_scatter(x=xs,y=Ts, name='bubble', mode='lines', row=1,col=1)
fig.add_scatter(x=y1s,y=Ts, name='dew', mode='lines', row=1,col=1)
fig.add_scatter(x=xs,y=y1s, mode='lines', row=1,col=2, name='y1')
fig.add_scatter(x=[0,1],y=[0,1], mode='lines', row=1,col=2, name='', line_color='grey')
fig.update_layout(width=800,height=400, title=f'Isopropanol-Water NRTL VLE at P={P} Pa', xaxis_title='x, y', yaxis_title='Temperature (K)')

In [19]:
x1Akima = sp.interpolate.Akima1DInterpolator(y1s, xs)
TyAkima = sp.interpolate.Akima1DInterpolator(y1s, Ts)
TxAkima = sp.interpolate.Akima1DInterpolator(xs, Ts)

Hvs = []
Hls = []
zs = np.linspace(0,0.999,50)
for z in zs:
    Hvs.append(p.Hv(np.array([z, 1-z]), TyAkima(z)))
    Hls.append(p.Hl(np.array([z, 1-z]), TxAkima(z)))  #could add additional term for p.Hex enthalpy of mixing
HvAkima = sp.interpolate.Akima1DInterpolator(zs, Hvs)
HlAkima = sp.interpolate.Akima1DInterpolator(zs, Hls)


In [20]:
def flashNRTL_PT(z, P, T):
    dewTres, dewxres=dewT_NRTL(z, P)
    bubbleTres, bubbleyres = bubbleT_NRTL(z, P)
    x0 = (T - dewTres)*(z - dewxres)/(bubbleTres - dewTres) + dewxres
    for i in range(100):
        # print(i, x0)
        K = p.NRTL_gamma(x0,T)*p.Pvap(T)/P

        def rr0(phi):
            return np.sum(z*(K-1)/(1+phi*(K-1)))

        # res = sp.optimize.root_scalar(rr0, bracket=(0,1))
        res = sp.optimize.root_scalar(rr0, x0=0.5, method='newton')
        # print(res)
        phi=res.root
        x = z/(1+phi*(K-1))
        if np.linalg.norm(x-x0)<1e-6:
            break
        x0 = x
    y = x*p.NRTL_gamma(x,T)*p.Pvap(T)/P
    return (phi, x, y)


In [21]:
def flashNRTL_PQ(z, feedH, flashP, Q=0):

    def energybalance(flashT):
        phires, xres, yres = flashNRTL_PT(z, flashP, flashT)
        liquidH = F*(1-phires)*p.Hl(xres, flashT)
        vaporH = F*phires*p.Hv(yres,flashT)
        return liquidH + vaporH - feedH - Q

    res = sp.optimize.root_scalar(energybalance, bracket=(bubbleT_flashP,dewT_flashP), method='bisect')
    flashT = res.root
    phires, xres, yres = flashNRTL_PT(z, flashP, flashT)
    return (flashT, phires, xres, yres)




In [22]:
#Vn1*Hvn1 = Ln*Hln + D*Hd + Qc
#(Ln + D)* Hvn1 = Ln * Hln + D*Hd + Qc
#Ln(Hvn1 - Hl) = D*Hd + Qc - D*Hvn1


#Ln/D *(Hvn1 - Hln) = Hd + Qc/D - Hvn1
#Ln/D*(yn1 - xn) = xd - yn1

In [31]:
p = Props(['Isopropanol','Water'])
F = 1.
z = 0.3
q=1.
Hf = p.Hl(np.array([z, 1-z]), TxAkima(z))

rD = 0.95  #recovery of isopropanol in distillate product
xD = 0.62
D = F*z*rD/xD
B = F - D
xB = F*z*(1-rD)/B
Hb = p.Hl(np.array([xB, 1-xB]), TxAkima(xB))

R=0.5
V1 = D*(R+1)


#Qc/D = V1*(Hv1 - Hd)/D
Hd = p.Hl(np.array([xD, 1-xD]), TxAkima(xD))
Qc = V1*(p.Hv(np.array([xD, 1-xD]), TyAkima(xD))- Hd)
Qb = D*Hd + B*Hb + Qc - F*Hf

In [32]:
#Ln/D *(Hvn1 - Hln) = Hd + Qc/D - Hvn1
#Ln/D*(yn1 - xn) = xD - yn1
# (Hd + Qc/D - Hvn1)/(Hvn1 - Hln) - (xD - yn1)/(yn1 - xn) = 0
def EMRec(yn1, Hln, xn):
    Hvn1 = p.Hv(np.array([yn1, 1-yn1]), TyAkima(yn1))
    return (Hd + Qc/D - Hvn1)*(yn1 - xn) - (xD - yn1)*(Hvn1 - Hln)

def EMStrip(yn1, Hln, xn):
    Hvn1 = p.Hv(np.array([yn1, 1-yn1]), TyAkima(yn1))
    return (Hln - (Hb - Qb/B))*(yn1-xn) - (xn - xB)*(Hvn1 - Hln)


In [34]:
yn1s=[xD]
xn1s=[]
yn1=xD
for i in range(100):
    xn1 = x1Akima(yn1).item()
    xn1s.append(xn1)
    if xn1<xB:
        break
    Hl = p.Hl(np.array([xn1, 1-xn1]), TxAkima(xn1))
    print(i,xn1)
    if xn1>z:
        yn1 = sp.optimize.root_scalar(lambda yn1: EMRec(yn1, Hl, xn1), bracket=(xn1,yn1)).root
    else:
        yn1 = sp.optimize.root_scalar(lambda yn1: EMStrip(yn1, Hl, xn1), bracket=(xn1,yn1)).root
    yn1s.append(yn1)


0 0.5674855055564247
1 0.5205547143486929
2 0.46987528188196953
3 0.398466973741908
4 0.2324242332952662
5 0.04670284623297177


In [42]:
fig=make_subplots()
fig.add_scatter(x=zs, y=Hvs, mode='lines',name='Hvap')
fig.add_scatter(x=zs, y=Hls, mode='lines',name='Hliq')
fig.add_scatter(x=(xD,xD),y=(Hd, Hd+Qc/D), mode='lines', line_color='green')

first_line = False
for (x,y) in zip(xn1s, yn1s):
    fig.add_scatter(x=(x,y),y=(p.Hl(np.array([x, 1-x]), TxAkima(x)), p.Hv(np.array([y, 1-y]), TyAkima(y))), mode='lines', line_dash='dot', line_color='rgb(100, 100, 100)')
    if x>z:
        fig.add_scatter(x=(x,xD),y=(p.Hl(np.array([x, 1-x]), TxAkima(x)), Hd+Qc/D), mode='lines', line_color='green')
    else:
        if first_line == False:
            first_line = True
        else:
            fig.add_scatter(x=(y,xB),y=(p.Hv(np.array([y, 1-y]), TyAkima(y)), Hb-Qb/B), mode='lines', line_color='green')

fig.update_xaxes(range=[0,xD+0.05])
fig.update_layout(width=600, height=600, showlegend=False)